In [24]:
import json
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from scipy.special import stirling2
import math

In [2]:
status_codes = {
    1: 'LOADED',
    2: 'OPTIMAL',
    3: 'INFEASIBLE',
    4: 'INF_OR_UNBD',
    5: 'UNBOUNDED',
    6: 'CUTOFF',
    7: 'ITERATION_LIMIT',
    8: 'NODE_LIMIT',
    9: 'TIME_LIMIT',
    10: 'SOLUTION_LIMIT',
    11: 'INTERRUPTED',
    12: 'NUMERIC',
    13: 'SUBOPTIMAL',
    14: 'INPROGRESS',
    15: 'USER_OBJ_LIMIT'
}

In [3]:
rng = np.random.default_rng(8)

# SDP Moment Equations for Independent Birth Death

Tidy 'initial_work' exploration into full setup of feasibility test for a choice of highest moment (order) d

## Code

In [15]:
def compute_A(alpha, d):
    '''
    Moment equation coefficient matrix
    NOTE: must have alpha <= d

    Args:
        alpha: moment order for equation (d/dt mu^alpha = 0)
        d: maximum moment order used

    Returns:
        A: (2, d + 1) matrix of coefficients
    '''

    if alpha > d:
        raise NotImplementedError("Maximum moment order 'd' too small for order 'alpha' moment equation: involves moments of order higher than d.")

    x = sp.Symbol('x')
    
    # reaction settings
    R = 2

    # reaction propensity polynomials
    props = [
        1,
        x
    ]

    # reaction vectors
    vrs = [
        1,
        -1
    ]

    # number of moments of order <= d
    Nd = d + 1

    # setup matrix
    A = np.zeros((R, Nd))

    for r, prop in enumerate(props):

        # expand b(x) * ((x + v_r)**alpha - x**alpha)
        poly = sp.Poly(prop * ((x + vrs[r])**alpha - x**alpha), x)

        # collect coefficients
        coeffs = poly.all_coeffs()

        # reverse
        coeffs.reverse()

        # put coefficients in matrix
        for l in range(Nd):

            try:
                coeff = coeffs[l]
            except IndexError:
                coeff = 0

            A[r, l] = coeff

    return A

In [16]:
def compute_B(beta, d):
    '''
    Capture efficiency moment scaling matrix

    Args:
        beta: per cell capture efficiency sample
        d: maximum moment order used

    Returns:
        B: (d + 1, d + 1) matrix of coefficients
    '''

    # compute beta moments
    y_beta = np.zeros(d + 1)
    for l in range(d + 1):
        y_beta[l] = np.mean(beta**l)

    n, p = sp.symbols('n p')

    def falling_factorial(n, k):
        val = 1
        for i in range(k):
            val *= (n - i)
        return val

    def binomial_moment(l):
        ''''''
        val = 0
        for k in range(l + 1):
            val += falling_factorial(n, k) * stirling2(l, k) * p**k
        return val

    # setup matrix
    B = np.zeros((d + 1, d + 1))

    # for each moment
    for l in range(d + 1):

        # compute expression
        moment = binomial_moment(l)
        poly = sp.Poly(moment, n, p)

        # collect coefficients
        for (j, k), coeff in zip(poly.monoms(), poly.coeffs()):
            B[l, j] += coeff * y_beta[k]

    return B

In [ ]:
def construct_M_0(y, d):
    '''Moment matrix variable constructor (s = 0).'''
    D = math.floor(d / 2)
    M_0 = [[] for i in range(D + 1)]
    for i in range(D + 1):
        for j in range(D + 1):
            M_0[i].append(y[i + j].item())
    M_0 = gp.MVar.fromlist(M_0)
    return M_0

In [87]:
def construct_M_1(y, d):
    '''Moment matrix variable constructor (s = 1).'''
    D = math.floor((d - 1) / 2)
    M_1 = [[] for i in range(D + 1)]
    for i in range(D + 1):
        for j in range(D + 1):
            M_1[i].append(y[i + j + 1].item())
    M_1 = gp.MVar.fromlist(M_1)
    return M_1

In [101]:
def base_model(model, OB_bounds, beta, d, time_limit=300):
    '''
    Construct 'base model' with semidefinite constraints removed to give NLP

    Args:
        model: empty gurobi model object
        OB_bounds: confidence intervals on observed moments up to order d
        beta: capture efficiency vector
        d: maximum moment order used
        time_limit: optimization time limit

    Returns:
        model: gurobi model object with NLP constraints (all but semidefinite)
        variables: dict for model variable reference
    '''

    # model settings
    model.Params.TimeLimit = time_limit
    K = 100

    # variables
    y = model.addMVar(shape=(d + 1), vtype=GRB.CONTINUOUS, name="y", lb=0)
    k = model.addMVar(shape=2, vtype=GRB.CONTINUOUS, name="k", lb=0, ub=K)

    # moment matrices
    M_0 = construct_M_0(y, d)
    M_1 = construct_M_1(y, d)
    
    # constraints

    # confidence interval bounds on OB moments
    y_lb = OB_bounds[0, :]
    y_ub = OB_bounds[1, :]

    # B scaling matrix
    B = compute_B(beta, d)

    # moment bounds
    model.addConstr(B @ y <= y_ub, name="y_UB")
    model.addConstr(B @ y >= y_lb, name="y_LB")

    # moment equations
    for alpha in range(d + 1):
        A_alpha_d = compute_A(alpha, d)
        model.addConstr(k.T @ A_alpha_d @ y == 0, name=f"ME_{alpha}_{d}")

    # base
    model.addConstr(y[0] == 1, name="y0_base")

    # fixed parameter
    model.addConstr(k[1] == 1, name="k2_fixed")

    # variable dict
    variables = {
        'y': y,
        'k': k,
        'M_0': M_0,
        'M_1': M_1
    }

    return model, variables

In [89]:
def optimize(model):
    '''Optimize model with no objective, return status.'''

    # optimize
    model.setObjective(0, GRB.MINIMIZE)
    model.optimize()
    status = status_codes[model.status]

    return model, status

In [90]:
def semidefinite_cut(model, variables, print_evals=False):
    '''
    Check semidefinite feasibility of NLP feasible point
    Feasible: stop
    Infeasible: add cutting plane

    Args:
        model: optimized NLP model
        variables: model variable reference dict
        print_evals: option to display moment matrix eigenvalues (semidefinite condition)

    Returns:
        model: model with any cutting planes added
        bool: semidefinite feasibility status
    '''

    # get moment matrix values
    M_0_val = variables['M_0'].X
    M_1_val = variables['M_1'].X

    # check semidefinite
    evals_0, evecs_0 = np.linalg.eigh(M_0_val)
    evals_1, evecs_1 = np.linalg.eigh(M_1_val)

    if print_evals:
        print("Moment matices eigenvalues:")
        print(evals_0)
        print(evals_1)

    # positive eigenvalues
    if (evals_0 >= 0).all() and (evals_1 >= 0).all():

        print("SDP feasible\n")
    
        return model, True

    # negative eigenvalue
    else:

        print("SDP infeasible")

        if evals_0[0] < 0:

            # get eigenvector
            v = evecs_0[0]

            # add cutting plane
            model.addConstr(v.T @ variables['M_0'] @ v >= 0, name="Cut_0")

            print("M_0 cut added\n")

        if evals_1[0] < 0:

            # get eigenvector
            v = evecs_1[0]

            # add cutting plane
            model.addConstr(v.T @ variables['M_1'] @ v >= 0, name="Cut_1")

            print("M_1 cut added\n")

    return model, False

In [146]:
def feasibility_test(OB_bounds, beta, d):
    '''
    Full feasibility test of birth death model via following algorithm

    Optimize NLP
    Infeasible: stop
    Feasible: check SDP feasibility
        Feasible: stop
        Infeasible: add cutting plane and return to NLP step

    Args:
        OB_bounds: confidence intervals on observed moments up to order d
        beta: capture efficiency vector
        d: maximum moment order used

    Returns:
        None
    '''

    # silent
    options = {'OutputFlag': 0}

    # environment context
    with gp.Env(params=options) as env:

        # model context
        with gp.Model('test-SDP', env=env) as model:

            # construct base model (no semidefinite constraints)
            model, variables = base_model(model, OB_bounds, beta, d)

            # check feasibility
            model, status = optimize(model)

            # while feasible
            while status == "OPTIMAL":

                print("NLP feasible")

                # check semidefinite feasibility
                model, semidefinite_feas = semidefinite_cut(model, variables, print_evals=True)

                # semidefinite feasible
                if semidefinite_feas:
                    break

                # semidefinite infeasible
                else:

                    # check feasibility with added cut
                    model, status = optimize(model)

            # if infeasible
            if status == "INFEASIBLE":
                print("SDP infeasible")

In [147]:
def bootstrap(sample, d, N=1000):
    '''
    Estimate bootstrap intervals of sample moments up to order d

    Args:
        sample: integer sample of length n
        d: maximum moment order to estimate
        N: number of bootstrap samples

    Returns:
        y_bounds: (2, d + 1) array of moment confidence intervals
    '''

    n = len(sample)

    # resample
    resamples = rng.choice(sample, size=(N, n))

    # estimate
    y_bounds = np.zeros((2, d + 1))
    for l in range(d + 1):
        
        moment_estimates = np.mean(resamples**l, axis=1)

        moment_interval = np.quantile(moment_estimates, [0.025, 0.975])

        y_bounds[:, l] = moment_interval

    return y_bounds

## Test

In [148]:
# settings
k1, k2 = 3, 1
b = 2
n = 1000
N = 1000
d = 3

# sample
sample = rng.poisson(k1 / k2, size=n)

# downsample
beta = rng.beta(1, b, size=1000)
downsample = rng.binomial(sample, beta, N)

# bootstrap
y_bounds = bootstrap(downsample, d)

# test feasibility
feasibility_test(y_bounds, beta, d)

NLP feasible
Moment matices eigenvalues:
[ 0.23099333 13.33113601]
[ 0.46097003 63.34715358]
SDP feasible



## Interaction test

In [149]:
from interaction_inference.simulation import gillespie_birth_death

In [ ]:
# settings
k1 = 5
k2 = 1
k_reg = 3
b = 1
n = 1000
N = 1000

# sample
params = {
    'k_tx_1': k1,
    'k_tx_2': k1,
    'k_deg_1': k2,
    'k_deg_2': k2,
    'k_reg': k_reg
}
sample = gillespie_birth_death(params, n)
sample = [x[0] for x in sample]

# downsample
if b == 0:
    beta = np.ones(n)
else:
    beta = rng.beta(1, b, size=1000)
downsample = rng.binomial(sample, beta, N)

# mean expression level
print(f"Mean expression {np.mean(downsample)}")

Mean expression 0.739


In [181]:
# settings
d = 3

# bootstrap
y_bounds = bootstrap(downsample, d)

# test feasibility
feasibility_test(y_bounds, beta, d)

SDP infeasible


## Telegraph test

In [169]:
from interaction_inference.simulation import gillespie_telegraph

In [199]:
# settings
k_on = 1
k_off = 1
k1 = 5
k2 = 1
k_reg = 0
b = 1
n = 1000
N = 1000

# sample
params = {
    'k_on_1': k_on,
    'k_on_2': k_on,
    'k_off_1': k_off,
    'k_off_2': k_off,
    'k_tx_1': k1,
    'k_tx_2': k1,
    'k_deg_1': k2,
    'k_deg_2': k2,
    'k_reg': k_reg
}
sample = gillespie_telegraph(params, n)
sample = [x[0] for x in sample]

# downsample
if b == 0:
    beta = np.ones(n)
else:
    beta = rng.beta(1, b, size=1000)
downsample = rng.binomial(sample, beta, N)

# mean expression level
print(f"Mean expression {np.mean(downsample)}")

Mean expression 1.262


In [200]:
# settings
d = 3

# bootstrap
y_bounds = bootstrap(downsample, d)

# test feasibility
feasibility_test(y_bounds, beta, d)

SDP infeasible


## Notes

- decent performance
- d > 3 typically neccessary to detect infeasible data
- for higher d may be issues when low mean expression levels

- only testing $X_{1}$ from a pair, results would improve if $X_{2}$ also checked for independence